In [1]:
import pandas as pd

from src.coingecko import CoingeckoAPI
from src.the_graph import TheGraphAPI

In [2]:
# get current Compound Borrows from The Graph

tg_api = TheGraphAPI()
borrows = tg_api.get_compound_borrows()
borrows_df = pd.DataFrame(borrows)
borrows_df

,underlyingSymbol,underlyingName,borrowRate,totalBorrows,collateralFactor,underlyingPriceUSD,blockTimestamp,accrualBlockNumber
0,USDP,Pax Dollar,0.0331804492674912,19732.997768734245967033,0,1,1669003091,16015853
1,TUSD,TrueUSD,0.0324051448416768,5487390.95791731311009598,0,1,1669000295,16015620
2,REP,Reputation,0.0304073961368256,55.908821869668995077,0,6.9611,1667094731,15857708
3,UNI,Uniswap,0.041839994188944,430209.110581609193362697,0.75,5.382985,1669043567,16019205
4,USDC,USD//C,0.0188994962998656,268116530.872985,0.855,1,1669046015,16019409
5,SUSHI,SushiToken,0.0687260633849568,622043.32507834035424997,0.7,1.16,1669035779,16018559
6,ETH,Ether,0.022934568919248,13700.010751671939880926,0.825,1134.535768,1669044263,16019260
7,DAI,Dai Stablecoin,0.0233026004452608,211524161.923877439677332104,0.835,1.000773,1669045535,16019369
8,BAT,Basic Attention Token,0.0208749131542368,444566.374199117545566316,0.65,0.2213,1669039655,16018881
9,COMP,Compound,0.0257408160614208,10248.127213140608051478,0.65,38.037452,1669036751,16018640


Have two `wBTC` rows... only keep the bigger one for now
```
{
  "underlyingSymbol": "WBTC",
  "underlyingName": "Wrapped BTC",
  "borrowRate": "0.0206864728473312",
  "totalBorrows": "0.39387562",
  "collateralFactor": "0.7",
  "underlyingPriceUSD": "15929.996902",
  "blockTimestamp": 1669001867,
  "accrualBlockNumber": 16015751
},
{
  "underlyingSymbol": "WBTC",
  "underlyingName": "Wrapped BTC",
  "borrowRate": "0.0221761103560704",
  "totalBorrows": "152.21575285",
  "collateralFactor": "0.7",
  "underlyingPriceUSD": "16182.771321",
  "blockTimestamp": 1669043411,
  "accrualBlockNumber": 16019192
},
```

In [3]:
# get historical prices from coingecko

cg_api = CoingeckoAPI()
coin_info = pd.DataFrame(cg_api.get_coins_list())

In [4]:
# drop wormholes and bridges

mask = coin_info["id"].apply(lambda s: ("-wormhole" not in s) & ("bridge" not in s))
print(f"dropping: {coin_info['id'][~mask].tolist()}")
coin_info = coin_info[mask]

dropping: ['1sol-io-wormhole', 'aave-wormhole', 'ageur-wormhole', 'allbridge', 'audius-wormhole', 'avalanche-wormhole', 'axie-infinity-shard-wormhole', 'barnbridge', 'binance-coin-wormhole', 'bitcoin-avalanche-bridged-btc-b', 'bridge', 'bridgecoin-2', 'bridge-mutual', 'bridge-network', 'bridge-oracle', 'bridgesplit-brand-index', 'bridgetech-usdt', 'celo-euro-wormhole', 'chainlink-wormhole', 'constitutiondao-wormhole', 'cross-chain-bridge', 'decentraland-wormhole', 'dogelon-mars-wormhole', 'dydx-wormhole', 'ethereum-wormhole', 'force-bridge-usdc', 'ftx-wormhole', 'gravity-bridge-dai', 'gravity-bridge-tether', 'gravity-bridge-usdc', 'gravity-bridge-wbtc', 'gravity-bridge-weth', 'hash-bridge-oracle', 'lido-dao-wormhole', 'luna-wormhole', 'matic-wormhole', 'metabridge', 'nix-bridge-token', 'octus-bridge', 'orbit-bridge-klaytn-binance-coin', 'orbit-bridge-klaytn-ethereum', 'orbit-bridge-klaytn-matic', 'orbit-bridge-klaytn-ripple', 'orbit-bridge-klaytn-usdc', 'orbit-bridge-klaytn-usd-tether'

In [8]:
borrows_df["underlyingSymbol"] = borrows_df["underlyingSymbol"].apply(lambda s: s.lower())

tmp = borrows_df.merge(coin_info, how="left", left_on="underlyingSymbol", right_on="symbol")
tmp


,underlyingSymbol,underlyingName,borrowRate,totalBorrows,collateralFactor,underlyingPriceUSD,blockTimestamp,accrualBlockNumber,id,symbol,name
0,usdp,Pax Dollar,0.0331804492674912,19732.997768734245967033,0,1,1669003091,16015853,paxos-standard,usdp,Pax Dollar
1,usdp,Pax Dollar,0.0331804492674912,19732.997768734245967033,0,1,1669003091,16015853,usdp,usdp,USDP Stablecoin
2,tusd,TrueUSD,0.0324051448416768,5487390.95791731311009598,0,1,1669000295,16015620,true-usd,tusd,TrueUSD
3,rep,Reputation,0.0304073961368256,55.908821869668995077,0,6.9611,1667094731,15857708,augur,rep,Augur
4,uni,Uniswap,0.041839994188944,430209.110581609193362697,0.75,5.382985,1669043567,16019205,unicorn-token,uni,UNICORN
5,uni,Uniswap,0.041839994188944,430209.110581609193362697,0.75,5.382985,1669043567,16019205,uniswap,uni,Uniswap
6,usdc,USD//C,0.0188994962998656,268116530.872985,0.855,1,1669046015,16019409,usd-coin,usdc,USD Coin
7,sushi,SushiToken,0.0687260633849568,622043.32507834035424997,0.7,1.16,1669035779,16018559,sushi,sushi,Sushi
8,eth,Ether,0.022934568919248,13700.010751671939880926,0.825,1134.535768,1669044263,16019260,ethereum,eth,Ethereum
9,dai,Dai Stablecoin,0.0233026004452608,211524161.923877439677332104,0.835,1.000773,1669045535,16019369,dai,dai,Dai
